In [127]:
%load_ext autoreload
%autoreload 2

import src.models.gps as gps
import src.data.loader as loader
from torch import set_printoptions, ones, arange
from sklearn.model_selection import train_test_split

set_printoptions(threshold=10000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
from torch_geometric.datasets import ExplainerDataset, graph_generator
from torch_geometric.explain import Explainer, GNNExplainer, ModelConfig

In [129]:
data, num_classes = loader.load_clean_bashapes()

In [132]:
model = gps.GPS(data, num_classes, pe_channels=5, num_layers=3, hidden_channels=8, observe_attention=True)
gps.train(model, data)

100%|██████████| 100/100 [02:44<00:00,  1.64s/it]


[tensor([[[0.0014, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0014],
          [0.0014, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0014],
          [0.0014, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0014],
          ...,
          [0.0014, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0014],
          [0.0014, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0014],
          [0.0014, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0014]]],
        grad_fn=<MeanBackward1>),
 tensor([[[0.0000e+00, 1.4013e-45, 1.2892e-43,  ..., 1.2612e-43,
           9.5288e-44, 1.2331e-43],
          [2.9178e-16, 2.0009e-14, 6.0143e-14,  ..., 6.5523e-14,
           6.2253e-14, 6.5006e-14],
          [1.2937e-06, 2.4853e-06, 2.9365e-06,  ..., 3.2655e-06,
           3.2857e-06, 3.2516e-06],
          ...,
          [4.2364e-03, 1.9308e-03, 1.5690e-03,  ..., 1.4085e-03,
           1.4103e-03, 1.4101e-03],
          [3.7268e-03, 1.9149e-03, 1.6062e-03,  ..., 1.4139e-03,
           1.4095e-03, 1.4158e-03],
          [4.3345e-03, 1.9433e-03

In [136]:
gps.test(model, data)

(0.48392857142857143, 0.45714285714285713)

In [160]:
gnn_explainer = Explainer(
        model=model,
        algorithm=GNNExplainer(epochs=300),
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
        model_config=ModelConfig(
            mode='multiclass_classification',
            task_level='node',
            return_type='raw',
        )
    )

explanation = gnn_explainer(data.x, data.edge_index, index=3, laplacian_eigenvector_pe=data.laplacian_eigenvector_pe)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [174]:
explanation

Explanation(node_mask=[700, 1], edge_mask=[3940], prediction=[700, 4], target=[700], index=[1], x=[700, 1], edge_index=[2, 3940], laplacian_eigenvector_pe=[700, 5])